# Access Control

<a href="http://35.236.121.59/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fproject-chip%2Fconnectedhomeip&urlpath=lab%2Ftree%2Fconnectedhomeip%2Fdocs%2Fguides%2Frepl%2FMatter%2520-%2520Access%2520Control.ipynb&branch=master">
<img src="https://i.ibb.co/hR3yWsC/launch-playground.png" alt="drawing" width="130"/>
</a>
<br></br>

This document explains how to use Access Control in Matter, and will be updated as development proceeds.

## What Does and Doesn’t Work Right Now?

Briefly, you can read and write the entire ACL attribute in the all-clusters-app, but Access Control isn’t yet turned on, so it won’t affect interactions. There’s almost no error checking when writing the ACL attribute (e.g. ensuring subjects match auth mode, only your fabric can be written, etc.) so exercise caution for now.

## Clear Persisted Storage

Let's clear out our persisted storage (if one exists) to start from a clean slate.

In [7]:
import os, subprocess

if os.path.isfile('/tmp/repl-storage.json'):
    os.remove('/tmp/repl-storage.json')

# So that the all-clusters-app won't boot with stale prior state.
os.system('rm -rf /tmp/chip_*')

0

## Initialization

Let's first begin by setting up by importing some key modules that are needed to make it easier for us to interact with the Matter stack.

> **NOTE**: _This is not needed if you launch the REPL from the command-line._

In [8]:
import chip.native
import pkgutil
module = pkgutil.get_loader('chip.ChipReplStartup')
%run {module.path}

──────────────────────────────────────── Matter REPL ────────────────────────────────────────

    
            Welcome to the Matter Python REPL!
    
            For help, please type matterhelp()
    
            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.
    
            

─────────────────────────────────────────────────────────────────────────────────────────────

Restoring FabricAdmin from storage to manage FabricId 1, FabricIndex 1...

New FabricAdmin: FabricId: 1(1)


Fabric Admins have been loaded and are available at fabricAdmins

Creating default device controller on fabric 1...

Allocating new controller with FabricId: 1(1), NodeId: 1


Default CHIP Device Controller has been initialized to manage fabricAdmins[0], and is 
available as devCtrl

## Commission and Setup Server

### Launch Server

Let's launch an instance of the `chip-all-clusters-app`.

In [9]:
import time, os
import subprocess
os.system('pkill -f chip-all-clusters-app')
time.sleep(1)

# The location of the all-clusters-app in the cloud playground is one level higher - adjust for this by testing for file presence.
if (os.path.isfile('../../../out/debug/chip-all-clusters-app')):
    appPath = '../../../out/debug/chip-all-clusters-app'
else:
    appPath = '../../../../out/debug/chip-all-clusters-app'

process = subprocess.Popen(appPath, stdout=subprocess.DEVNULL)
time.sleep(1)

### Commission Target

Commission the target with a NodeId of 1.

In [10]:
devCtrl.CommissionIP(b'127.0.0.1', 20202021, 2)

2022-01-29 16:01:43 johnsj-macbookpro1.roam.corp.google.com chip.SC[9915] ERROR The device does not support GetClock_RealTimeMS() API. This will eventually result in CASE session setup failures.


Node address has been updated
Commissioning complete


True

## Bootstrap ACLs

(For now) normally after commissioning there would be at least a single admin entry, but currently the ACL will be empty, so add that entry manually. This step will be removed later when it’s no longer necessary.

In [11]:
await devCtrl.ReadAttribute(2, [ (0, Clusters.OperationalCredentials)], True)

{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: OperationalCredentials(
│   │   │   NOCs=[
│   │   │   │   NOCStruct(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   noc=b'\x150\x01\x01\x01$\x02\x017\x03$\x13\x01$\x15\x01\x18&\x04\x80"\x81\'&\x05\x80%M:7\x06$\x11\x02$\x15\x01\x18$\x07\x01$\x08\x010\tA\x04\x19\x1ef\xaa\x8d_A\x19\x9f\xee\xa1\x1fn\x82\xb6}(\xd6\x00\'\xeb\x80Y\xf9\xd7\xa8\xbe\x98\xce\xba4v\xa0\xa0\xa9\xbaUO)?\xbb\xe0\xa4\x12r\x0cb\xe0\xc8\xa7r\xe3\xd5\x8e\x159\xc6\xaf\xc8\xbc\xb4\x8b\xf9E7\n5\x01(\x01\x18$\x02\x016\x03\x04\x02\x04\x01\x180\x04\x14\x1f\x87b\xe9`2\x03C{o`\x9e\x14\xe3\x8c\x0b\x83\xcd\x10Z0\x05\x14p\xfc\xf9\nFI\xba9\xb6SH\xcb\xd0meg\n\t\\=\x180\x0b@\xd3\x03x<\x0f\xdc\xceBl\x01\xfb[Kly+\xd4\xab\xa08\xd3f \xa5\xb9\xb4aP\xce|\xee"to\x17\xc1m\xcfQ\x10pq=7\r\xf6\xe4\x89+\xcb\x98\xa9\x176T\x82\x83\x8d\x04#\xa2\xd3\xf2\xb6\x18',
│   │   │   │   │   icac=b'\x150\x01\x01\x00$\x02\x017\x03$\x14\x00$\x15\x01\x18&\x04\x80"\x81\'&\x05\x80%M:7\x06$\x13\x01$\x15\x01\x18$\x07\x01$\x08\x010\tA\x04,\x0c\x82d\xa8\xd8-o\x860Q\xeb\x8d\x87\xe6x\x9e\x0b\xf8\xc91\xa9\xc5\x01nxB\x17h\xbc\x98]\xc9&\x19\x9f\xde\x97&\x80M\xca\x8b\xa1\xa0g\xfd\xae}\x12\x8a\x98\x08\x86k\xc7=\xc3\xea\x0e\xb0\xf0p\x057\n5\x01)\x01\x18$\x02`0\x04\x14p\xfc\xf9\nFI\xba9\xb6SH\xcb\xd0meg\n\t\\=0\x05\x14~\xf0\n\'\'\x89\xc26\xa8\xdaz\xbeCv\xb2\x7fn\xca^\xb5\x180\x0b@\x11\xa5\xf9\x80\x1a\x9c\xe5\xbf\x1e\xb3\n\x83\x0eW6!\x0f\xb0,\xa7\xdb\xb0\xe3B\xdc\\\xec\xbb\x02\xc2\x04\xd9\x0c\xa7\xdfj\x1a\x15]\x18\x08\xdfFz%)b\xfa\x9b\xca\x99\xccY\x01\xa7}\xf1|\xe6\xfe\xf0*\x10W\x18'
│   │   │   │   )
│   │   │   ],
│   │   │   fabricsList=[
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   rootPublicKey=b'\x04\xa2\x88}\xbd\xdb\xd4\xbc^U\xa3\xcef\xf7\x92\xbe\xa7X;E\xb8-Y\x12^$\xe0\x1bS\xf9\x9a\x8f\xacvj\xfa!&m6\x00\xe3\xd1\x85o\x1b\x9eH\xabC?\xd8\xec\x08lH\xb5X\x1f:u\xb0\xfbj\x98',
│   │   │   │   │   vendorId=31968,
│   │   │   │   │   fabricId=1,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label=''
│   │   │   │   )
│   │   │   ],
│   │   │   supportedFabrics=16,
│   │   │   commissionedFabrics=1,
│   │   │   trustedRootCertificates=[
│   │   │   │   b'\x150\x01\x01\x00$\x02\x017\x03$\x14\x00$\x15\x01\x18&\x04\x80"\x81\'&\x05\x80%M:7\x06$\x14\x00$\x15\x01\x18$\x07\x01$\x08\x010\tA\x04\xa2\x88}\xbd\xdb\xd4\xbc^U\xa3\xcef\xf7\x92\xbe\xa7X;E\xb8-Y\x12^$\xe0\x1bS\xf9\x9a\x8f\xacvj\xfa!&m6\x00\xe3\xd1\x85o\x1b\x9eH\xabC?\xd8\xec\x08lH\xb5X\x1f:u\xb0\xfbj\x987\n5\x01)\x01\x18$\x02`0\x04\x14~\xf0\n\'\'\x89\xc26\xa8\xdaz\xbeCv\xb2\x7fn\xca^\xb50\x05\x14~\xf0\n\'\'\x89\xc26\xa8\xdaz\xbeCv\xb2\x7fn\xca^\xb5\x180\x0b@3;\xd5n\xa0\xb1O\xecI:\x9cM\xf4u\x12a\x96\x1d\x13\x19\xa4D\xb3v&_o.(\xcbs\xeb\xc0\xc4\xb80\xc2\xecF4\xfbV\'\xf7X\xe2A\xaa\xa5l\r5\xba\xbd\xa4I&C\xff\xed\xd8\xa8_\x06\x18'
│   │   │   ],
│   │   │   currentFabricIndex=1,
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=1
│   │   )
│   }
}

In [5]:
acl = [ Clusters.AccessControl.Structs.AccessControlEntryStruct(
    fabricIndex = 1,
    privilege = Clusters.AccessControl.Enums.AccessControlEntryPrivilegeEnum.kAdminister,
    authMode = Clusters.AccessControl.Enums.AccessControlEntryAuthModeEnum.kCase,
    subjects = [ 1 ] ) 
]

acl

[
│   AccessControlEntry(
│   │   fabricIndex=1,
│   │   privilege=<Privilege.kAdminister: 5>,
│   │   authMode=<AuthMode.kCase: 2>,
│   │   subjects=[
│   │   │   1
│   │   ],
│   │   targets=Null
│   )
]

In [6]:
await devCtrl.WriteAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl( acl ) ) ] )

[
│   AttributeStatus(
│   │   Path=AttributePath(
│   │   │   EndpointId=0,
│   │   │   ClusterId=31,
│   │   │   AttributeId=0
│   │   ),
│   │   Status=<Status.Success: 0>
│   )
]

In [7]:
data = await devCtrl.ReadAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl) ] )
data

{
│   0: {
│   │   <class 'chip.clusters.Objects.AccessControl'>: {
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Acl'>: [
│   │   │   │   AccessControlEntry(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   privilege=<Privilege.kAdminister: 5>,
│   │   │   │   │   authMode=<AuthMode.kCase: 2>,
│   │   │   │   │   subjects=[
│   │   │   │   │   │   1
│   │   │   │   │   ],
│   │   │   │   │   targets=Null
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

In [8]:
acl = data[0][chip.clusters.Objects.AccessControl][chip.clusters.Objects.AccessControl.Attributes.Acl]
acl

[
│   AccessControlEntry(
│   │   fabricIndex=1,
│   │   privilege=<Privilege.kAdminister: 5>,
│   │   authMode=<AuthMode.kCase: 2>,
│   │   subjects=[
│   │   │   1
│   │   ],
│   │   targets=Null
│   )
]

In [9]:
acl.append(Clusters.AccessControl.Structs.AccessControlEntryStruct(
    fabricIndex = 1,
    privilege = Clusters.AccessControl.Enums.AccessControlEntryPrivilegeEnum.kOperate,
    authMode = Clusters.AccessControl.Enums.AccessControlEntryAuthModeEnum.kCase,
    targets = [ Clusters.AccessControl.Structs.AccessControlTargetStruct(
        endpoint = 1,
    ) ] ) )
acl

[
│   AccessControlEntry(
│   │   fabricIndex=1,
│   │   privilege=<Privilege.kAdminister: 5>,
│   │   authMode=<AuthMode.kCase: 2>,
│   │   subjects=[
│   │   │   1
│   │   ],
│   │   targets=Null
│   ),
│   AccessControlEntry(
│   │   fabricIndex=1,
│   │   privilege=<Privilege.kOperate: 3>,
│   │   authMode=<AuthMode.kCase: 2>,
│   │   subjects=Null,
│   │   targets=[
│   │   │   Target(
│   │   │   │   cluster=Null,
│   │   │   │   endpoint=1,
│   │   │   │   deviceType=Null
│   │   │   )
│   │   ]
│   )
]

In [10]:
await devCtrl.WriteAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl( acl ) ) ] )

[
│   AttributeStatus(
│   │   Path=AttributePath(
│   │   │   EndpointId=0,
│   │   │   ClusterId=31,
│   │   │   AttributeId=0
│   │   ),
│   │   Status=<Status.Success: 0>
│   )
]

In [11]:
await devCtrl.ReadAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl ) ] )

{
│   0: {
│   │   <class 'chip.clusters.Objects.AccessControl'>: {
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Acl'>: [
│   │   │   │   AccessControlEntry(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   privilege=<Privilege.kAdminister: 5>,
│   │   │   │   │   authMode=<AuthMode.kCase: 2>,
│   │   │   │   │   subjects=[
│   │   │   │   │   │   1
│   │   │   │   │   ],
│   │   │   │   │   targets=Null
│   │   │   │   ),
│   │   │   │   AccessControlEntry(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   privilege=<Privilege.kOperate: 3>,
│   │   │   │   │   authMode=<AuthMode.kCase: 2>,
│   │   │   │   │   subjects=Null,
│   │   │   │   │   targets=[
│   │   │   │   │   │   Target(
│   │   │   │   │   │   │   cluster=Null,
│   │   │   │   │   │   │   endpoint=1,
│   │   │   │   │   │   │   deviceType=Null
│   │   │   │   │   │   )
│   │   │   │   │   ]
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}